In [11]:
import pandas as pd
import twint
import numpy as np
import nest_asyncio
nest_asyncio.apply()
import datetime
import ast
from newspaper import Article
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

In [12]:
df = pd.read_csv("NepaliTimes_analysis.csv", parse_dates=['created_at'])

In [13]:
list(df)

['id',
 'created_at',
 'urls',
 'article',
 'level1',
 'level2',
 'level3',
 'level1_count',
 'level_2_3_valid',
 'level2_count',
 'level3_count']

In [14]:
def clean_text(raw):
    text = BeautifulSoup(raw, 'lxml').text
    word_tokens = word_tokenize(text.lower().rstrip())
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return ' '.join(filtered_sentence)

In [15]:
def article(url):
    if (len(url)==0):
        return 404
    url = url[0]
    try:
        article = Article(url)
        article.download()
        article.parse()
        return clean_text(article.text)
    except Exception as e:
        return 404

In [16]:
def get_new_articles(newspaper, since, until):
    c = twint.Config()
    c.Username = newspaper
    c.Pandas = True
    c.Hide_output= True
    c.Since = since
    c.Until = until
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    df = df[['id', 'created_at', "urls"]]
    return df
#     df['article'] = df.urls.apply(article)
#     df = df[df['article']!=404]
#     return df

In [ ]:
df_ = get_new_articles('NepaliTimes', since=df.created_at.max().strftime("%Y-%m-%d %H:%M:%S"), until=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

In [ ]:
df_['article'] = df_.urls.apply(article)

In [ ]:
df